### Mount(Montar) Azure Data Lake para el Proyecto

1. Obtener el valor client_id, tenant_id y client_secret del key Vault
2. Configurar Spark con APP/Client Id, Directory/Tenant Id & Secret
3. Utiliza el método "mount" de "utility" para montar el almacenamiento
4. Explorar otras utilidades del sistema de archivos relacionados con el montaje (list all mounts, unmounts)

In [0]:
def mount_adls(storage_account_name, container_name):
    #Obtener el Secret Key de Key Vault
    client_id = dbutils.secrets.get(scope="movie-history-secret-scope", key="client-id")
    #directory
    tenant_id = dbutils.secrets.get(scope="movie-history-secret-scope", key="tenand-id")
    client_secret = dbutils.secrets.get(scope="movie-history-secret-scope", key="client-secret")

    #Establecer configuraciones de Spark
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #Desmontar(unmount) el montaje(mount) si ya existe
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    #Mount(Montar) el contenedor de storage account
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",
    extra_configs = configs)

    #Listar los Mounts
    display(dbutils.fs.mounts())

### Montar(mount) en el contenedor "bronze"

In [0]:
mount_adls("moviehistory0001", "bronze")

### Montar(mount) en el contenedor silver

In [0]:
mount_adls("moviehistory0001", "silver")

### Montar(mount) en el contenedor gold

In [0]:
mount_adls("moviehistory0001", "gold")

### Montar(mount) en el contenedor "demo"

In [0]:
mount_adls("moviehistory0001", "demo")